PARTIE 1 — Initialisation du projetInitialisation 

In [25]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

In [8]:
# chargement du dataset
df = pd.read_csv('../data/events.csv')
df.head()
df.info()
df['event'].value_counts(normalize=False)
df['event'].value_counts(normalize=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64


PARTIE 2 — Nettoyage & Exploration

In [9]:
# filter uniquement vieuw et add_to_cart
df_ab = df[df['event'].isin(['view', 'add_to_cart'])].copy

In [11]:
# valeurs manquantes
print(df.isna().sum())

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64


In [12]:
# nombre d'evenements par type dans ce sous-ensemble
print(df["event"].value_counts())
print(df["event"].value_counts(normalize=True))

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64
event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64


In [13]:
# nomnre d'users uniques
n_visiteurs = df["visitorid"].nunique()
print("Visiteurs uniques :", n_visiteurs)

Visiteurs uniques : 1407580


PARTIE 3 — Simulation d’un A/B Test

In [14]:
# visiteurs unique 
visitors = df["visitorid"].drop_duplicates()
len(visitors)

1407580

In [15]:
#  Assigner A/B aléatoirement (50/50)
np.random.seed(42)  # Pour la reproductibilité
assignments = np.random.choice(["A", "B"], size=len(visitors), p=[0.5, 0.5])

visitor_groups = pd.DataFrame({
    "visitorid": visitors.values,
    "group": assignments
})

visitor_groups["group"].value_counts(normalize=True)


group
A    0.500024
B    0.499976
Name: proportion, dtype: float64

In [16]:
# ajouter la colonne 'group' au dataset
df = df.merge(visitor_groups, on="visitorid", how="left")
df[["visitorid", "group"]].head()
df["group"].value_counts(normalize=True)

group
A    0.500737
B    0.499263
Name: proportion, dtype: float64

PARTIE 4 — KPI : Add-to-Cart Rate

In [17]:
# compter view et add_to_cart par groupe
summary = (
    df.groupby(["group", "event"])["visitorid"]
      .count()
      .unstack(fill_value=0)
)

summary = summary.rename(columns={"view": "n_views", "addtocart": "n_addtocart"})
summary


event,n_addtocart,transaction,n_views
group,,,
A,35217,11794,1333072
B,34115,10663,1331240


In [18]:
# calculer le taux de add_to_cart
summary["add_to_cart_rate"] = summary["n_addtocart"] / summary["n_views"]
summary

event,n_addtocart,transaction,n_views,add_to_cart_rate
group,,,,
A,35217,11794,1333072,0.026418
B,34115,10663,1331240,0.025626


Interpretation pour le KPI

le groupe A est supérieur au groupe B



In [30]:
# La différence en points de pourcentage
rate_A = summary.loc["A", "add_to_cart_rate"]
rate_B = summary.loc["B", "add_to_cart_rate"]
diff_pp = (rate_A - rate_B) * 100
print("Différence (A - B) en points de % :", diff_pp)

Différence (A - B) en points de % : 0.07914443346517426


PARTIE 5 — Test statistique : Test de proportion

In [20]:
# recupérer les nombres clés
summary
summary
# index : group ("A", "B")
# colonnes : "n_views", "n_addtocart", "add_to_cart_rate"

n1 = summary.loc["A", "n_views"]        # taille échantillon A
x1 = summary.loc["A", "n_addtocart"]    # "succès" = addtocart A
n2 = summary.loc["B", "n_views"]
x2 = summary.loc["B", "n_addtocart"]

p1 = x1 / n1
p2 = x2 / n2
print("p_A :", p1)
print("p_B :", p2)


p_A : 0.02641792791387112
p_B : 0.025626483579219376


In [21]:
# proportion combinée
p_pooled = (x1 + x2) / (n1 + n2)
print("Proportion combinée :", p_pooled)

Proportion combinée : 0.02602247784793973


In [22]:
# z-score
se = np.sqrt(p_pooled * (1 - p_pooled) * (1/n1 + 1/n2))
z = (p2 - p1) / se
print("z-score :", z)


z-score : -4.057271085129678


In [26]:
# p-value (bilatéral)
p_value = 2 * (1 - norm.cdf(abs(z)))
print("p-value :", p_value)


p-value : 4.964944166507834e-05


In [27]:
# décision au seuil de alpha = 0.05
alpha = 0.05
if p_value < alpha:
    print("On rejette H0 : différence significative entre A et B.")
else:
    print("On ne rejette pas H0 : pas de différence significative au seuil 5%.")

On rejette H0 : différence significative entre A et B.


PARTIE 6 - Analyse Business

- Différence entre les taux

Le groupe A affiche un taux d'engagement au panier approximativement de 2,64%, tandis que le groupe B enregistre 2,56%, ce qui représente une différence approximative de 0,08 point de pourcentage favorable à A. Bien que ce décalage soit numériquement minime, il signale que la version A produit un peu plus d'ajouts au panier par rapport à la version B, pour une quantité équivalente de vues.

- Significativité statisque

Le test de proportion pour deux échantillons indique un z-score approximatif de −4,06 et une p-value autour de 4,96×10^−5, largement sous le seuil α=0,05. Cela implique que la différence constatée entre les taux d'ajout au panier de A et B a une chance très minime d'être due au hasard, et qu'on peut affirmer l'existence d'une différence statistiquement significative entre les deux options.

- Risque d'erreur

En utilisant un seuil de signification α=0,05, on admet nécessairement un risque de 5% de commettre une erreur de première espèce, c'est-à-dire refuser l'hypothèse nulle lorsqu'elle est effectivement vraie. Dans ce contexte spécifique, la valeur p étant nettement inférieure à 0,05, le danger de tirer une conclusion erronée sur l'existence d'une différence, tout en respectant cette limite, semble minime compte tenu de la puissance du signal statistique.

- Décision produit

D'un angle commercial, la variante B montre un taux d'ajout au panier légèrement inférieur à A, et cette diminution est statistiquement pertinente. Bien que l'impact soit faible en termes absolus, la mise en œuvre de B entraînerait une détérioration quantifiable d'un indicateur clé associé à l'intention d'achat, sans avantage discernable sur d'autres paramètres. Donc, la suggestion est de s'abstenir de mettre en production la variante B et de garder A comme version principale. Si l'intention est d'améliorer encore le parcours, il serait judicieux de concevoir et tester une variante C qui apporterait des modifications plus significatives (design, contenu, offre) et possiblement un bénéfice plus conséquent, à examiner grâce à un autre test A/B.
